In [3]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import lightgbm as lgbm

In [4]:
data_date = 221201

In [5]:
room_info = pd.read_csv(f'data_preprocessed/room_info_{data_date}.csv')

In [10]:
# room_subway_df = pd.read_csv(f'data_subway/room_subway_station_dist_{data_date}.csv')
# data = pd.concat([room_info, room_subway_df], axis=1)

In [12]:
data = room_info.copy()

In [13]:
charter_info = data[data['sales_type'] == '전세'].reset_index(drop=True)
monthly_info = data[data['sales_type'] == '월세'].reset_index(drop=True)

In [14]:
charter_info = charter_info[charter_info['보증금액']<= 50000].reset_index(drop=True)

## Modeling
- 전세 매물에 대해 보증금액을 예측하는 모델

In [15]:
train_cols = ['전용면적_m2', 'approve_year', 'random_lat', 'random_long', 'is_underground']\
#     + room_subway_df.columns[1:].tolist()

In [16]:
X = charter_info[train_cols]
y = charter_info['보증금액']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [17]:
model = lgbm.LGBMRegressor()
model.fit(X_train, y_train);
pred = model.predict(X_test)

In [18]:
r2_score(y_test, pred)

0.7655152076032833

## 전세 월세 gap
- 1000만원에 5만원

In [19]:
pred2 = model.predict(monthly_info[train_cols])
monthly_info['전세금액_pred'] = pred2
ratio = (monthly_info['전세금액_pred'] - monthly_info['보증금액'])/monthly_info['월세금액']
monthly_info['ratio']  = ratio
monthly_info[monthly_info['보증금액'] < 5000].sort_values('ratio')

,item_id,user_no,sales_type,보증금액,월세금액,전용면적_m2,공급면적_m2,local1,local2,local3,...,floor,updated_at,approve_date,residence_type,approve_year,random_lat,random_long,is_underground,전세금액_pred,ratio
2069,34332556,17569039,월세,1300,1300.0,130.89,193.94,서울시,강남구,도곡동,...,NaN,2022-11-30 16:39:04,2002-10-23,아파트/업무시설,2002,37.487761,127.053220,0,40288.601040,29.991232
2074,34338383,16442359,월세,1300,1300.0,130.89,281.18,서울시,강남구,도곡동,...,NaN,2022-12-01 10:00:55,20021023,오피스텔,2002,37.487745,127.053245,0,40288.601040,29.991232
2769,34012292,27598,월세,3000,43.0,18.00,18.00,서울시,강북구,수유동,...,반지하,2022-11-06 10:05:01,20151208,다중주택,2015,37.635890,127.019550,1,4741.516509,40.500384
6211,34260468,12653,월세,2000,25.0,16.53,16.53,서울시,금천구,가산동,...,6,2022-11-25 12:18:58,20080401,제2종근린생활시설,2008,37.472959,126.887411,0,3125.347192,45.013888
7075,34226591,17174521,월세,1000,790.0,85.76,160.70,서울시,강남구,논현동,...,NaN,2022-11-22 17:46:18,20070130,오피스텔,2007,37.516789,127.039804,0,43011.001238,53.178483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11290,33995790,5952091,월세,3000,15.0,26.45,26.45,서울시,관악구,신림동,...,1,2022-11-04 15:35:28,20180614,다중주택,2018,37.466442,126.938454,0,15181.287688,812.085846
15370,34338640,944159,월세,3000,9.0,18.84,18.84,서울시,구로구,구로동,...,6,2022-12-01 10:09:01,20061214,제2종근린생활시설,2006,37.482811,126.899532,0,11193.360052,910.373339
11577,34356067,4944581,월세,3000,34.0,58.45,58.45,서울시,송파구,삼전동,...,4,2022-12-01 18:29:24,20171128,다세대주택,2017,37.504457,127.091073,0,36699.298660,991.155843
2362,33956876,13501011,월세,1500,5.0,26.00,26.00,서울시,도봉구,쌍문동,...,반지하,2022-11-02 10:30:20,1989-05-29,단독주택,1989,37.649802,127.032475,1,7413.903666,1182.780733


In [20]:
monthly_info.sort_values('ratio')

,item_id,user_no,sales_type,보증금액,월세금액,전용면적_m2,공급면적_m2,local1,local2,local3,...,floor,updated_at,approve_date,residence_type,approve_year,random_lat,random_long,is_underground,전세금액_pred,ratio
7430,34131104,13903,월세,13200,2.0,23.14,26.45,서울시,강서구,화곡동,...,NaN,2022-11-28 20:56:51,2012.11.09,도시형생활주택,2012,37.545111,126.846639,0,8743.847680,-2228.076160
6741,34189441,17981377,월세,16000,3.0,12.92,12.92,서울시,성동구,마장동,...,3,2022-11-22 16:45:15,20120711,다세대(도시형생활주택),2012,37.563085,127.043229,0,10495.051683,-1834.982772
3849,34189427,17981377,월세,16000,3.0,12.09,12.09,서울시,성동구,마장동,...,4,2022-11-19 13:15:32,20120711,도시형생활주택,2012,37.563109,127.043222,0,10846.896365,-1717.701212
13893,34165629,1157271,월세,20000,5.0,23.14,23.14,서울시,관악구,봉천동,...,8,2022-11-17 15:59:37,20220720,오피스텔,2022,37.480157,126.953030,0,14686.473177,-1062.705365
13520,34299424,80202,월세,15000,2.0,23.14,23.14,서울시,동작구,사당동,...,3,2022-11-28 14:16:49,"2017. 3,2",공동주택,2017,37.478423,126.976987,0,13015.195620,-992.402190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7541,34333071,9099358,월세,14000,3.0,29.80,49.59,서울시,영등포구,당산동,...,NaN,2022-11-30 17:01:44,2011-03-23,오피스텔,2011,37.526986,126.908610,0,21261.172090,2420.390697
13433,34007685,2636502,월세,29000,5.0,51.61,61.08,서울시,서초구,방배동,...,2,2022-11-05 15:01:15,2016.4.1,다세대주택,2016,37.487489,126.988955,0,41186.950586,2437.390117
3721,34178491,2362010,월세,8670,2.0,18.34,18.34,서울시,중구,장충동2가,...,1,2022-11-18 14:39:29,19980911,다세대주택,1998,37.559554,127.001044,0,13742.963782,2536.481891
3719,33979410,7591389,월세,8670,2.0,18.34,18.34,서울시,중구,장충동2가,...,2,2022-11-21 17:22:26,1998-09-11,공동주택,1998,37.559538,127.001678,0,13742.963782,2536.481891


In [21]:
ratio.mean()

211.06365350565505

In [22]:
monthly_info[['보증금액', '월세금액', '전세금액_pred']]

,보증금액,월세금액,전세금액_pred
0,500,50.0,9283.019364
1,500,43.0,13963.141060
2,500,40.0,9283.019364
3,12000,30.0,15691.094538
4,3000,58.0,14103.342070
...,...,...,...
16072,1000,45.0,9456.665200
16073,1000,55.0,17055.365831
16074,1000,45.0,13738.855063
16075,3000,52.0,12483.192505


## 좋은 전세 매물

In [23]:
charter_info_pred = charter_info.copy()

In [24]:
charter_info_pred['pred'] = model.predict(X)

In [25]:
charter_info_pred['score'] = charter_info_pred['pred'] - charter_info_pred['보증금액']  # 높을수록 좋음

In [26]:
def select_dong(df, wide=[], narrow=[]):
    result = pd.DataFrame()
    for i in wide:
        result = pd.concat([result, df[df['local3'].str.contains(i)]])
    for i in narrow:
        result = pd.concat([result, df[df['local3'] == i]])
    result = result.reset_index(drop=True)
    return result

### 동 기준

In [27]:
df = select_dong(charter_info_pred, ['문래', '당산', '망원'])

### 지하철 기준

In [ ]:
subway = pd.read_csv(f'./data_subway/subway_info_{data_date}.csv')

In [38]:
from haversine import haversine

In [79]:
def select_by_station(df, stations=[]):
    result = pd.DataFrame()
    for station in stations:
        sw = subway[subway['name'] == station]
        df_near = df[df.apply(lambda x:haversine((float(x['random_lat']), float(x['random_long'])), 
                            (sw['lat'].values[0], sw['lng'].values[0]), unit='m') < 800, axis=1)].copy()
        result = pd.concat([result, df_near])
    return result.sort_values('score', ascending=False)

In [119]:
sbs = select_by_station(df, ['영등포구청역', '영등포시장역', '신길역', '대방역', '합정역', '망원역'])

### 대출

In [88]:
from glob import glob
from tqdm import tqdm
import pickle

In [89]:
target_date = '221201'
data_filenames = glob('data_room/*')
data_filenames = [x for x in data_filenames if target_date in x]
data = []
for fn in tqdm(data_filenames):
    with open(fn, 'rb') as f:
        data.extend(pickle.load(f)) 

100%|██████████| 56/56 [00:01<00:00, 29.04it/s]


In [120]:
sbs = sbs[sbs['보증금액'] < 20000]

In [121]:
sbs_top = sbs[sbs['score'] > 2000]

In [110]:
loan_info = []
for d in data:
    if re.findall('안심전세|버팀목|보험|중기청', d['item']['description'].replace(' ', '')):
        loan_info.append((d['item']['item_id'], d['item']['description']))
loan_info = pd.DataFrame(loan_info)
loan_info.columns = ['item_id', 'description']

In [123]:
m = pd.merge(sbs_top, loan_info)

In [126]:
m.iloc[0]['description']

'〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓\r\n\r\n💥💥 중기청80%, 버팀목, 카카오뱅크, 청년맞춤형대출 등 전세자금대출 가능합니다! 💥💥\r\n\r\n〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓\r\n\r\n◼ 대 중 교 통 ◼  \r\n\r\n- 영등포구청역, 영등포시장역 도보 5분 거리[2호선,5호선] 대박 거리\r\n- 타임스퀘어 도보 10분 주변 교통 수단 이용 가능합니다.\r\n\r\n◼  간 단 설 명 ◼\r\n\r\n🔥⚡영등포구청역 역세권에 귀한 전월세대출 가능한 방입니다. 🔥⚡\r\n\r\n- 신축 두번째 입주 화이트톤 인테리어!! \r\n   보시면 너무 예뻐서 감탄하셔요\r\n\r\n- 트인 뷰🐥 채광 촤르르🦊\r\n\r\n- 풀옵션! 에어컨, 세탁기, 대형 냉장고, 전자레인지, 인덕션, 신발장 등.\r\n\r\n- 안심 귀가 가능합니다~^^ 대로변에 근접하며 유동인구 많은 지역! 여자분들이 더 많이 거주하시는 건물!\r\n\r\n- 엘리베이터가 있어서 편리하며, 현관보안, cctv로 보안 책임집니다!!!!\r\n\r\n- 안전하고 젠틀한 집주인으로 스트레스 없이 거주하다, 안전하게 퇴실하세요!\r\n\r\n〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓\n 22.11.21에 직방에 등록된 매물입니다.'

In [127]:
m

,item_id,user_no,sales_type,보증금액,월세금액,전용면적_m2,공급면적_m2,local1,local2,local3,...,updated_at,approve_date,residence_type,approve_year,random_lat,random_long,is_underground,pred,score,description
0,34208977,9644013,전세,13000,NaN,17.52,17.52,서울시,영등포구,당산동1가,...,2022-11-21 14:18:48,2021.08.12,단독주택,2021,37.523150,126.900109,0,19256.295311,6256.295311,"〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓\r\n\r\n💥💥 중기청80%, 버팀목, 카카오..."
1,34208977,9644013,전세,13000,NaN,17.52,17.52,서울시,영등포구,당산동1가,...,2022-11-21 14:18:48,2021.08.12,단독주택,2021,37.523150,126.900109,0,19256.295311,6256.295311,"〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓\r\n\r\n💥💥 중기청80%, 버팀목, 카카오..."
2,34348095,9644013,전세,13000,NaN,19.83,19.83,서울시,영등포구,당산동1가,...,2022-12-01 14:16:38,2021-08-12,단독주택,2021,37.522653,126.900694,0,18096.743468,5096.743468,◇ + + + + + + + + + + + + + + ...
3,34348095,9644013,전세,13000,NaN,19.83,19.83,서울시,영등포구,당산동1가,...,2022-12-01 14:16:38,2021-08-12,단독주택,2021,37.522653,126.900694,0,18096.743468,5096.743468,◇ + + + + + + + + + + + + + + ...
4,34200474,9644013,전세,15500,NaN,20.50,20.50,서울시,영등포구,당산동3가,...,2022-11-27 10:25:11,2021.04.29,단독주택,2021,37.524627,126.896788,0,20313.800563,4813.800563,"〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓\r\n🌈 중기청80, 버팀목, 카뱅, 청년맞춤형..."
5,34200474,9644013,전세,15500,NaN,20.50,20.50,서울시,영등포구,당산동3가,...,2022-11-27 10:25:11,2021.04.29,단독주택,2021,37.524627,126.896788,0,20313.800563,4813.800563,"〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓\r\n🌈 중기청80, 버팀목, 카뱅, 청년맞춤형..."
6,34349233,5825625,전세,17000,NaN,29.80,49.65,서울시,영등포구,당산동,...,2022-12-01 16:45:07,2011-03-23,오피스텔,2011,37.526330,126.908622,0,21261.172090,4261.172090,"청~아한 중개사와 함께 보시죠 :D\r\n\r\n\r\n+ 중기청, 모든전세대출, ..."
7,33960091,18105989,전세,19000,NaN,31.00,51.00,서울시,영등포구,당산동,...,2022-11-02 12:23:58,20120806,오피스텔,2012,37.526396,126.908792,0,22036.191128,3036.191128,영등포시장역과 7분 거리이며 실평수가 10평이 넘어 2명도 거주 할 수 있는 크기 ...


## Tuning

In [77]:
def k_fold_evaluation(model, X, y, seed=42):
    cv = KFold(n_splits=5, shuffle=True, random_state=seed)
    cv_scores = []
    for idx, (train_idx, test_idx) in tqdm(enumerate(cv.split(X, y))):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(
          X_train,
          y_train,
        )
        preds = model.predict(X_test)
        rmse = mean_squared_error(y_test, preds, squared=False)
        rsquare = r2_score(y_test, preds)
        cv_scores.append([rmse, rsquare])
    return cv_scores

In [76]:
import optuna